Title: Common Table Expressions
Slug: sql/common-table-expressions
Category: SQL
Tags: WITH, SELECT, FROM, LIMIT, OVER, PARTITION BY, ORDER BY, DESC, rank
Date: 2017-08-16
Modified: 2017-08-16

#### Load ipython-sql extension

In [1]:
# The 2 lines below prevent an error message from being displayed when we run %load_ext sql
import warnings
warnings.filterwarnings('ignore')

%load_ext sql
%config SqlMagic.feedback = False

#### Connect to the database

In [2]:
%%sql

postgresql://localhost/dvdrental

'Connected: None@dvdrental'

#### Using the `WITH` clause
We use the `WITH` keyword to create temporary tables just for the query we're currently working on. These are called Common Table Expressions (CTEs).

In the simple example below, we execute the query in the `WITH` clause and then return all rows from it.

In [3]:
%%sql

WITH rental_cte AS (
    SELECT
        r.rental_id
        , customer_id
        , r.return_date
    FROM
        rental r
    LIMIT
        10
)

SELECT
    *
FROM
    rental_cte rc

rental_id,customer_id,return_date
2,459,2005-05-28 19:40:33
3,408,2005-06-01 22:12:39
4,333,2005-06-03 01:43:41
5,222,2005-06-02 04:33:21
6,549,2005-05-27 01:32:07
7,269,2005-05-29 20:34:53
8,239,2005-05-27 23:33:46
9,126,2005-05-28 00:22:40
10,399,2005-05-31 22:44:21
11,142,2005-06-02 20:56:02


#### A common use for CTEs — ranking your results
As explained in the page on [Window Functions](../sql/ranking-results-with-window-functions.html), CTEs are useful when ranking our results.

Here we will find the last 3 films returned by a sample of customers.

In [4]:
%%sql

WITH rental_cte AS (
    SELECT
        r.rental_id
        , customer_id
        , r.return_date
        , rank() OVER(PARTITION BY r.customer_id ORDER BY r.return_date DESC) as rk
    FROM
        rental r
)

SELECT
    *
FROM
    rental_cte rc
WHERE
    rc.rk < 4
-- Show only the first 3 customers to save space
LIMIT
    9

rental_id,customer_id,return_date,rk
15315,1,2005-08-30 01:51:46,1
15298,1,2005-08-28 22:49:37,2
14825,1,2005-08-27 07:01:57,3
15145,2,2005-08-31 15:51:04,1
14743,2,2005-08-29 00:18:56,2
14475,2,2005-08-27 08:59:32,3
14699,3,2005-08-29 18:08:48,1
13403,3,2005-08-27 19:23:07,2
15619,3,2005-08-26 07:21:14,3
